In [ ]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import time

#Ist time_since_last_change relevant
# Was machen wir, wenn change überfällig ist

In [ ]:
def calc_precision(preds,y,days_diff):
    return np.sum(np.abs(np.floor(preds)-y)<=days_diff)/len(y)

In [ ]:
df_pop= pd.read_csv('../../../data/popular_data_with_features3.csv', index_col=0)
df_pop.head()

In [ ]:
top_changes = df_pop.groupby(['infobox_key', 'property_name']).count()["day_of_year"].sort_values(ascending=False)

In [ ]:
buckets_sizes = [0,10,50,100,500,1000,99999]
buckets = []
for i in range(len(buckets_sizes)-1):
    low=buckets_sizes[i]
    high=buckets_sizes[i+1]
    keys=top_changes.loc[(top_changes>=low) & (top_changes<high)].index.to_list()
    buckets.append({"low":low,"high":high,"size":len(keys),"keys":keys})
buckets

In [ ]:
def generate_predictions(key_list):
    predictions=[]
    accuracys=[]
    feature_importances=[]
    for key in tqdm(key_list):
        sample = df_pop[(df_pop['infobox_key']==key[0]) & (df_pop['property_name']==key[1])]
        # sample = df_pop.iloc[start:start+iloc]
        trainsize=round(len(sample)*0.8) # 20 % testdata
        sample.drop(columns=["infobox_key","page_title","property_name","template"], inplace=True)

        X_train = sample.iloc[:trainsize].drop(columns=['days_until_next_change','timestamp'])

        y_train = sample.iloc[:trainsize]['days_until_next_change']

        X_test = sample.iloc[trainsize:].drop(columns=['days_until_next_change','timestamp'])
        y_test = sample.iloc[trainsize:]['days_until_next_change']
        # x_last = sample.iloc[-1].drop(['days_until_next_change','timestamp']).values.reshape(1,-1)
        
        # x_last_timestamp = datetime.datetime.strptime(sample.iloc[-1]["timestamp"],"%Y-%m-%d").date()
        clf = RandomForestClassifier(random_state=0, n_estimators=10)
        clf.fit(X_train, y_train)
        preds=clf.predict(X_test)
        predictions.append(preds)
        accuracys.append(calc_precision(preds,y_test,1))
        feature_importances.append(clf.feature_importances_)

    return predictions, accuracys, feature_importances

In [ ]:
import warnings
warnings.filterwarnings('ignore')
predictions=[]
accuracys=[]
feature_importances=[]
for key_list in tqdm(buckets[1:]):

    preds,accs,feature_imps=generate_predictions(key_list["keys"][:100])
    predictions.append(preds)
    accuracys.append(accs)
    feature_importances.append(feature_imps)


In [ ]:
import scipy
for i,acc in enumerate(accuracys):

    print("<",buckets[i+1]["high"],scipy.stats.describe(acc))

In [ ]:
columns=['day_of_year', 'day_of_month', 'day_of_week', 'month_of_year',
       'quarter_of_year', 'is_month_start', 'is_month_end', 'is_quarter_start',
       'is_quarter_end', 'days_since_last_change', 'days_since_last_2_changes',
       'days_since_last_3_changes', 'days_between_last_and_2nd_to_last_change',
       'mean_change_frequency_all_previous', 'mean_change_frequency_last_3']
feature_importances_df=pd.DataFrame(feature_importances,columns=columns)
feature_importances_df.describe()
feature_importances_df.to_csv("rf_feature_importances_1000rfs.csv")

In [ ]:
preds_df=pd.DataFrame(ilocs.index.to_list(),columns=["infobox_key","property_name"])
preds_df["pred"]=preds
preds_df